In [ ]:
%pip install flask pyngrok transformers scipy
from flask import Flask, request, send_file, make_response
from pyngrok import ngrok
import time
from transformers import pipeline
import scipy.io.wavfile
import os
import getpass

app = Flask(__name__)
authtoken = getpass.getpass("Ingresá tu NGROK_AUTHTOKEN (no se mostrará): ") 
ngrok.set_auth_token(authtoken)
public_url = ngrok.connect(5000)
print(f'El servidor está disponible públicamente en: {public_url}')

@app.route("/generate_music", methods=["POST", "OPTIONS"])
def generate_music():
    if request.method == 'OPTIONS':
          # Responder al preflight
          response = make_response()
          response.headers.add("Access-Control-Allow-Origin", "*")
          response.headers.add("Access-Control-Allow-Methods", "GET, POST, OPTIONS")
          response.headers.add("Access-Control-Allow-Headers", "Content-Type, Authorization")
          return response
    output_path = "output/musicgen_out.wav"
    prompt = request.json.get("prompt", "Default melody")  # Recibe el prompt del cliente
    start_time = time.time()

    try:
        print("Cargando modelo...")
        synthesiser = pipeline("text-to-audio", "facebook/musicgen-small")
        print("Modelo cargado. Generando música...")
        music = synthesiser(prompt, forward_params={"do_sample": True})
        scipy.io.wavfile.write(output_path, rate=music["sampling_rate"], data=music["audio"])
        print("Audio generado en %.2f segundos" % (time.time() - start_time))

        # Verifica si el archivo fue generado correctamente
        if os.path.exists(output_path):
            return send_file(output_path, as_attachment=True)
        else:
            return "Error: No se pudo generar el archivo de audio.", 500
    except Exception as e:
        print(f"Error: {e}")
        return f"Error al generar la música: {str(e)}", 500
                
# --- 6. Crear carpeta de salida si no existe ---,
if not os.path.exists("output"):
    os.makedirs("output")

# --- 7. Levantamos el servidor ---,
app.run(port=5000)

: 